In [1]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [52]:
my_date_handler = lambda year, month, day, **kwargs: "%04i-%02i-%02i" % (year, month, day)
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

### bring_conditoned_data


1. 드랍다운리스트 
    - rng.api.Validation.Delete()
    - rng.api.Validation.Add(Type=3, Formula1=state_list)

In [15]:
dv_type = xw.constants.DVType.xlValidateList
dv_alertstyle = xw.constants.DVAlertStyle.xlValidAlertStop
dv_operator = xw.constants.FormatConditionOperator.xlEqual

In [10]:
sel_sht = wb_cy.selection.sheet

In [18]:
# 셀정보받기
last_col = sel_sht.range('XFD9').end('left').column
last_row = sel_sht.range('A1048576').end('up').row
if last_row < 10 : last_row=10
sel_sht.range((10,1),(last_row,last_col)).delete() # 기존 데이터 삭제

col_names = sel_sht.range((9,1),(9,last_col)).value


# 드랍다운 리스트 받기

# DATE_TYPE
str_dt = 'Dog,Cat,Bat'
rng_dt = sel_sht.range((2,"C"))
rng_dt.api.Validation.Delete()
rng_dt.api.Validation.Add(Type=dv_type, Formula1=str_dt)

# # STATE
# str_state = "ALL," + ",".join(pd.DataFrame(cursor.execute("select * from inspection_code").fetchall())[0])
# rng_state = sel_sht.range((5,"C"))
# rng_state.api.Validation.Delete()
# rng_state.api.Validation.Add(Type=3, Formula1=str_state)

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, None, None, 0, -2146827284), None)

In [109]:
# 검색조건 받기
sel_sht = wb_cy.selection.sheet
conditions= []
for i in range(2,8):
    conditions.append(sel_sht.range((i,"C")).options(numbers=lambda e : str(int(e)), dates=my_date_handler).value)
    j=i-2
    if conditions[j] is None:
        if j == 0:
            conditions[j] = 'ARRIVAL_DATE'
        else:
            conditions[j] = 'ALL'
    

conditions

['ARRIVAL_DATE', 'ALL', '2023-03-02', 'SHIP_CONFIRM', 'ALL', 'KR_LTH']

In [110]:
base_query = "SELECT * FROM IR_ORDER "

if conditions[1] == 'ALL': conditions[1] = '1999-01-01'
if conditions[2] ==  'ALL': conditions[2] = '2199-12-31'

query = base_query + f"WHERE {conditions[0]} BETWEEN '{conditions[1]}' AND '{conditions[2]}' "
query = query if conditions[3] == 'ALL' else query + f"AND STATE = '{conditions[3]}' "
query = query if conditions[4] ==  'ALL' else query + f"AND ARTICLE_NUMBER = '{conditions[4]}' "
query = query if conditions[5] ==  'ALL' else query + f"AND SUBINVENTORY = '{conditions[5]}'"

# Data fomatting
df_ir = pd.DataFrame(cursor.execute(query).fetchall(),columns=col_names)
df_ir['TIMELINE'] = df_ir['TIMELINE'].map(lambda e: json.loads(e)['data'][-1]['c'])
df_ir = df_ir.sort_values('IR_INDEX')
df_ir.set_index('IR_INDEX',inplace=True)

sel_sht.range("A9").value = df_ir

last_row = sel_sht.range("B1048576").end('up').row
last_col = sel_sht.range("XFD9").end("left").column
xl_content = sel_sht.range((10,1),(last_row,last_col))
xl_content.api.Borders.LineStyle = 1 

In [107]:
def bring_condition_data_pod():
    
    
    wb_cy.app.screen_updating = False
    
    #셀정보받기
    last_row = sel_sht.range('A1048576').end('up').row
    if last_row < 10 : last_row=10
    sel_sht.range((10,1),(last_row,last_col)).delete() # 기존 데이터 삭제
    last_col = sel_sht.range('XFD9').end('left').column
    col_names = sel_sht.range((9,1),(9,last_col)).value


    # 드랍다운 리스트 받기

    # DATE_TYPE
    str_dt = 'ALL,ARRIVAL_DATE,SHIP_DATE'
    rng_dt = sel_sht.range((2,"C"))
    rng_dt.api.Validation.Delete()
    rng_dt.api.Validation.Add(Type=3, Formula1=str_dt)
    # STATE
    str_state = "ALL," + ",".join(pd.DataFrame(cursor.execute("select * from inspection_code").fetchall())[0])
    rng_state = sel_sht.range((5,"C"))
    rng_state.api.Validation.Delete()
    rng_state.api.Validation.Add(Type=3, Formula1=str_state)
    
    # 검색조건 받기
    sel_sht = wb_cy.selection.sheet
    conditions= []
    for i in range(2,8):
        conditions.append(sel_sht.range((i,"C")).options(numbers=lambda e : str(int(e)), dates=my_date_handler).value)
        j=i-2
        if conditions[j] is None:
            if j == 0:
                conditions[j] = 'ARRIVAL_DATE'
            else:
                conditions[j] = 'ALL'
                
    #조건값 가져오기
    base_query = "SELECT * FROM IR_ORDER "

    if conditions[1] == 'ALL': conditions[1] = '1999-01-01'
    if conditions[2] ==  'ALL': conditions[2] = '2199-12-31'

    query = base_query + f"WHERE {conditions[0]} BETWEEN '{conditions[1]}' AND '{conditions[2]}' "
    query = query if conditions[3] == 'ALL' else query + f"AND STATE = '{conditions[3]}' "
    query = query if conditions[4] ==  'ALL' else query + f"AND ARTICLE_NUMBER = '{conditions[4]}' "
    query = query if conditions[5] ==  'ALL' else query + f"AND SUBINVENTORY = '{conditions[5]}'"

    # 데이터프레임 형식에 맞게 조정
    df_ir = pd.DataFrame(cursor.execute(query).fetchall(),columns=col_names)
    df_ir['TIMELINE'] = df_ir['TIMELINE'].map(lambda e: json.loads(e)['data'][-1]['c'])
    df_ir = df_ir.sort_values('IR_INDEX')
    df_ir.set_index('IR_INDEX',inplace=True)
    
    # 데이터 엑셀로 전송
    sel_sht.range("A9").value = df_ir
    
    # 서식조정
    last_row = sel_sht.range("B1048576").end('up').row
    last_col = sel_sht.range("XFD9").end("left").column
    xl_content = sel_sht.range((10,1),(last_row,last_col))
    xl_content.api.Borders.LineStyle = 1 
    
    wb_cy.app.screen_updating = True

,IR_INDEX,ARTICLE_NUMBER,DESCRIPTION,QUANTITY,ORDER_NM,ARRIVAL_DATE,SUBINVENTORY,SHIP_DATE,COMMENT,STATE,TIMELINE
0,262,18111204,SEAL KIT 100ML,4,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,2023-03-01 11:10:27
1,263,18111126,"PH ELECTRODE, ROUND TIP",1,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,2023-03-01 11:10:27
2,264,29249817,LAMP ASSY CPL,1,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,2023-03-01 11:10:27
3,276,18112962,"UV TEST KIT 254NM, 2MM CELL",2,IR-SM00034308,2017-08-07,KR_LTH,2017-08-10,08/10 KR_SERV01로 Transfer후 stock,SHIP_CONFIRM,2023-03-01 11:10:27
4,278,28952645,P9S Drive unit,1,IR-SM00034389,2017-08-08,KR_LTH,2017-08-10,08/10삼성바이오에피스건으로 이태환과장님출고,SHIP_CONFIRM,2023-03-01 11:10:27
...,...,...,...,...,...,...,...,...,...,...,...
414,3692,BR100600,"System Check Kit, type 2",3,IR-SM00388508,2020-09-01,KR_LTH,2020-09-15,09/15 KR_SERV01로 stock,SHIP_CONFIRM,2023-03-01 11:10:27
415,3693,29256413,"PM kit HW, type105",3,IR-SM00388508,2020-09-01,KR_LTH,2020-09-15,09/15 KR_SERV01로 stock,SHIP_CONFIRM,2023-03-01 11:10:27
416,3969,52-852996-001-9,FRU MOTHERBOARD PCIE ASSY PROGRAMMED IC/MIC,1,IR-SM00401737,2020-10-13,KR_LTH,2020-10-14,10/14 이태환님 서울대 건으로 출고,SHIP_CONFIRM,2023-03-01 11:10:27
417,3723,28999471,PM-kit AKTA avant,1,IR-SM00386805,2020-09-04,KR_LTH,2020-09-15,09/15 이태환님 요청으로 KR_serv01로 stock,SHIP_CONFIRM,2023-03-01 11:10:27


In [98]:
df_ir

,0,1,2,3,4,5,6,7,8,9,10
0,262,18111204,SEAL KIT 100ML,4,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
1,263,18111126,"PH ELECTRODE, ROUND TIP",1,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
2,264,29249817,LAMP ASSY CPL,1,IR-SM00033782,2017-08-01,KR_LTH,2017-08-10,08/10녹십자건으로 이태환과장님출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
3,276,18112962,"UV TEST KIT 254NM, 2MM CELL",2,IR-SM00034308,2017-08-07,KR_LTH,2017-08-10,08/10 KR_SERV01로 Transfer후 stock,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
4,278,28952645,P9S Drive unit,1,IR-SM00034389,2017-08-08,KR_LTH,2017-08-10,08/10삼성바이오에피스건으로 이태환과장님출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
...,...,...,...,...,...,...,...,...,...,...,...
414,5877,18117773,PM-KIT AKTA PILOT,1,IR-SM00489467,2021-11-02,KR_LTH,2021-11-04,11/04 이태환님 요청으로 KR_SERV01로 이동,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
415,6499,11000322,CU-950 MEMORY KIT,2,IR-SM00495473,2022-01-20,KR_LTH,2022-02-15,02/15 이태환님 요청으로 출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
416,6500,28404168,KEYBOARD UK/US IP65,1,IR-SM00495473,2022-01-20,KR_LTH,2022-02-15,02/15 이태환님 요청으로 출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
417,5940,29273097,Atom board ver2/CFast 32 GB/battery,1,IR-SM00488627,2021-11-11,KR_LTH,2021-11-15,11/15 이태환님 출고,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."


In [48]:
sel_sht = wb_cy.selection.sheet
last_col = sel_sht.range('XFD9').end('left').column
col_names = sel_sht.range((9,1),(9,last_col)).value
idx_state=col_names.index('STATE')

12

In [51]:
len('GOOD_WR,GOOD_ELSE,DMG_WET,DMG_TEMPARTURE,DMG_ONLYOUTER,DMG_IMPACT,DMG_ELSE,WR_SHORT,WR_OVER,WR_PRODUCT,WR_ELSE,RETURN')

117

In [47]:
import string

print("\nAlphabet from A-Z:")
for i,letter in enumerate(string.ascii_uppercase):
    
       print(i,letter)


Alphabet from A-Z:
0 A
1 B
2 C
3 D
4 E
5 F
6 G
7 H
8 I
9 J
10 K
11 L
12 M
13 N
14 O
15 P
16 Q
17 R
18 S
19 T
20 U
21 V
22 W
23 X
24 Y
25 Z


In [56]:
def create_db_timeline(up_time_content=None):
    """
    DB Table의 col중 UP_TIME 같이 업데이트가 되는 테이블들은 해당 매서드로 업데이트 
    UPDATE 회수는 68번 제한 oracle varchar2()는 4000이 한계
    json.loads(obj)로 이용가능한 str return
    """
    import datetime as dt
    import json

    # create 시 처음 json데이터 생번
    data=[]
    cols=['a','b','c']
    if up_time_content is None:
        rows=[]
        now = str(dt.datetime.now()).split('.')[0]
        rows.append('create')
        rows.append(str(0))
        rows.append(now)
        tmp = dict(zip(cols,rows))
        data.append(tmp)
        res = {
            'meta':{
                'desc':'data_update_timeline',

                'cols':{
                    'a':'db_method',
                    'b':'count',
                    'c':'timeline',

                }
            },
            'data':data
        }
        return json.dumps(res,ensure_ascii=False)
    # 데이터 생성이후 update시마다 계속 업데이트
    else : 
        load_json = json.loads(up_time_content)['data']
        update_count=len(load_json)
        if len(load_json) == 68:
            return up_time_content
        rows=[]
        now = str(dt.datetime.now()).split('.')[0]
        rows.append('update')
        rows.append(str(update_count))
        rows.append(now)
        tmp = dict(zip(cols,rows))
        load_json.append(tmp)
        res = {
            'meta':{
                'desc':'data_update_timeline',

                'cols':{
                    'a':'db_method',
                    'b':'count',
                    'c':'timeline',

                }
            },
            'data':load_json
        }
        
        return json.dumps(res,ensure_ascii=False)

In [53]:
db_table_name = "IR_ORDER"
to_status = 'GOOD_WR'
pk_name = 'IR_INDEX'
val = 1

In [57]:
query = f"UPDATE {db_table_name} SET STATE = '{to_status}' WHERE {pk_name} = {val}"
bring_tl_query = f"SELECT TIMELINE FROM {db_table_name} WHERE {pk_name} = {val}"
try:
    json_tl = cursor.execute(bring_tl_query).fetchone()[0]
except:
    wb_cy.app.alert(f" {pk_name} : {val} 해당 INDEX는 DB에 등록되지 않은 INDEX입니다. Data는 DataInput기능으로만 저장 가능합니다. 종료합니다.","Quit")
timeline_query = f"UPDATE {db_table_name} SET TIMELINE = '{create_db_timeline(json_tl)}' WHERE {pk_name} = {val}"
cursor.execute(timeline_query)
cursor.execute(query)

In [58]:
cursor.execute('commit')